In [1]:
import pandas as pd
import numpy as np
import time
import datetime

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import sklearn as sk
from matplotlib import pyplot as plt
from scipy.sparse import csr_matrix
import scipy as sp
import re
import gensim
import logging
import seaborn as sns
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
%matplotlib inline
#import xgboost
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import time
from nltk.corpus import stopwords

train = pd.read_csv("data/features/features.csv",
                    header=0,delimiter=",",quotechar='"',error_bad_lines=False)

print("unfiltered data, num samples: %i with num features: %i" %(train.shape[0],train.shape[1]))

train.fillna(' ',inplace=True)
train.dropna(inplace=True)
print("filtered data with samples: %i" %(train.shape[0]))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
unfiltered data, num samples: 45885 with num features: 22
filtered data with samples: 45885


In [46]:
list(train)


['num',
 'record-number',
 'filename',
 'pdfismissing',
 'author',
 'fulltitle',
 'subtitle',
 'pages',
 'volume',
 'number',
 'keywords',
 'searchquery_terms',
 'classifications',
 'cancer_types',
 'label_top_level',
 'useful',
 'review_article',
 'not_allowed_in_germany',
 'year',
 'abstract',
 'pub-location',
 'publisher']

In [2]:
#y labels
useful = train["useful"]

#features
abstracts = train['abstract'].values.astype(dtype=str)
title = train['fulltitle'].values.astype(dtype=str)
journal = train['subtitle'].values.astype(dtype=str)

#only rows with abstract
idxthere = np.nonzero(train['abstract'] != ' ')[0]
title = title[idxthere]
journal = journal[idxthere]
abstracts = abstracts[idxthere]
useful = useful[idxthere]

In [6]:
journal = train['subtitle'].values.astype(dtype=str)
journal = journal[idxthere]

In [13]:
text[:1]

array([ 'To assess the effects of postoperative radiation therapy and chemotherapy on tumor recurrence and patient survival, 227 patients (data on 202 of whom were analyzed) who had undergone  curative  surgical resection for rectal adenocarcinoma were prospectively and randomly assigned to one of four treatments: no adjuvant therapy (concurrent controls, 58 patients), postoperative radiotherapy with 4000 or 4800 rad (50 patients), postoperative chemotherapy (fluorouracil and semustine [methyl-CCNU], 48 patients), or a combination of radiation therapy and chemotherapy (46 patients). Five years after the entry of the last patient and with a median follow-up of all survivors for 80 months, the recurrence rate was highest among the control patients (55 per cent) and lowest among the patients receiving a combination of adjuvant radiation and chemotherapy (33 per cent). Time to tumor recurrence differed significantly among the four treatment groups (P less than 0.04); it was significantly p

In [35]:
#one hot encoded
le = sk.preprocessing.LabelEncoder()
enc = sk.preprocessing.OneHotEncoder()
le_journal = le.fit_transform(journal)

le_journal = le_journal.reshape(le_journal.shape[0], 1)
hot_journal = enc.fit_transform(le_journal) 

In [38]:
start = time.time()
#preprocessing

#bag of words
reRemoved = [re.sub(r'[^a-z ]', '', s.lower()) for s in abstracts]
vect = CountVectorizer(max_df=0.6,min_df=10,stop_words=stopwords.words("english"))
bow_text = vect.fit_transform(reRemoved)

reRemoved = [re.sub(r'[^a-z ]', '', s.lower()) for s in title]
bow_title = vect.fit_transform(reRemoved)

In [48]:
print(type(hot_journal))

<class 'scipy.sparse.csr.csr_matrix'>


In [70]:
#only keep journals with more than 100 occurences
journal_count = np.sum(hot_journal, axis=0)
idx_imp_journals = np.nonzero(journal_count > 20)[0]
#print(idx_imp_journals.shape)
imp_journals = hot_journal[:,idx_imp_journals]
print(imp_journals)
#print(idx_imp_journals)

  (10, 368)	1.0
  (10, 367)	1.0
  (10, 366)	1.0
  (10, 365)	1.0
  (10, 364)	1.0
  (10, 363)	1.0
  (10, 362)	1.0
  (10, 361)	1.0
  (10, 360)	1.0
  (10, 359)	1.0
  (10, 358)	1.0
  (10, 357)	1.0
  (10, 356)	1.0
  (10, 355)	1.0
  (10, 354)	1.0
  (10, 353)	1.0
  (10, 352)	1.0
  (10, 351)	1.0
  (10, 350)	1.0
  (10, 349)	1.0
  (10, 348)	1.0
  (10, 347)	1.0
  (10, 346)	1.0
  (10, 345)	1.0
  (10, 344)	1.0
  :	:
  (39636, 24)	1.0
  (39636, 23)	1.0
  (39636, 22)	1.0
  (39636, 21)	1.0
  (39636, 20)	1.0
  (39636, 19)	1.0
  (39636, 18)	1.0
  (39636, 17)	1.0
  (39636, 16)	1.0
  (39636, 15)	1.0
  (39636, 14)	1.0
  (39636, 13)	1.0
  (39636, 12)	1.0
  (39636, 11)	1.0
  (39636, 10)	1.0
  (39636, 9)	1.0
  (39636, 8)	1.0
  (39636, 7)	1.0
  (39636, 6)	1.0
  (39636, 5)	1.0
  (39636, 4)	1.0
  (39636, 3)	1.0
  (39636, 2)	1.0
  (39636, 1)	1.0
  (39636, 0)	1.0


In [71]:
#combine features
features = sp.sparse.hstack((bow_title, bow_text, imp_journals))


In [65]:
print(features.shape)

(42868, 18749)


In [36]:
print(le_journal.shape)
print(type(le_journal))
print(hot_journal.shape)
len(np.unique(le_journal))
#print(bow_text.shape)
#print(bow.shape)

(42868, 1)
<class 'numpy.ndarray'>
(42868, 4768)


4768

In [72]:
import xgboost as xgb
# read in data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, useful, test_size=0.2, random_state=42)

model = xgb.XGBClassifier(max_depth=10,n_estimators=400)
model.fit(X_train, y_train)


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=None, n_estimators=400, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [73]:
preds = model.predict_proba(X_test)[:,1]
#print(preds[:10])
print(preds.shape)
print(y_test.shape)
#np.mean(np.power(preds - y_test,2))
sk.metrics.roc_auc_score(y_test,preds)

(8574,)
(8574,)


0.81843117411110922